In [1]:
get_ipython().system('pip install db-dtypes')

In [2]:
import os
import pandas
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics

In [3]:
BIGQUERY_PROJECT = 'ironhacks-data'
bigquery_client = bigquery.Client(project=BIGQUERY_PROJECT)

In [4]:
# get data information
query1 = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.unemployment_data`
"""
query2 = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.wage_data`
"""
query3 = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.prediction_list`
"""

In [5]:
query_job1 = bigquery_client.query(query1)
query_job2 = bigquery_client.query(query2)
query_job3 = bigquery_client.query(query3)

In [6]:
unemployment = query_job1.to_dataframe()
wage = query_job2.to_dataframe()
pred = query_job3.to_dataframe()

In [7]:
# combine all data
query4 = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.unemployment_data` as unem
inner join `ironhacks-data.ironhacks_competition.wage_data` as wage
on wage.uu_id = unem.uu_id
"""
query_job4 = bigquery_client.query(query4)
data = query_job4.to_dataframe()

In [8]:
# drop duplicate columns
dropcol = ['uu_id_1','countyfips_1','tract_1','tract_name_1']
for col in dropcol:
    data = data.drop(col, axis=1)

In [9]:
data.head()

,uu_id,timeperiod,week_number,countyfips,tract,tract_name,total_claims,edu_8th_or_less,edu_grades_9_11,edu_hs_grad_equiv,...,gender_male,gender_na,race_amerindian,race_asian,race_black,race_noanswer,race_hawaiiannative,race_other,race_white,average_wage
0,f013068de98db1470bd986137a0c6d23,20220416,16,18003,900,"Census Tract 9, Allen County, Indiana",22,0,<NA>,14,...,<NA>,0,0,0,11,0,0,<NA>,<NA>,8347.125000
1,21957d5517323845818d87623589e1ba,20220319,12,18089,10400,"Census Tract 104, Lake County, Indiana",111,0,<NA>,108,...,35,0,0,0,<NA>,<NA>,0,0,0,7036.636364
2,6a5609f385912113b6f1014b958ed748,20220326,13,18089,11500,"Census Tract 115, Lake County, Indiana",39,0,<NA>,<NA>,...,<NA>,0,0,0,39,0,0,0,0,7890.142857
3,46b2882ec4c373527ec33f7bd4f1388d,20220716,29,18089,20700,"Census Tract 207, Lake County, Indiana",14,0,<NA>,<NA>,...,<NA>,0,0,<NA>,10,0,0,<NA>,<NA>,7534.375000
4,37495d17e82f7df326bfc2c4c090f7b7,20220409,15,18089,21900,"Census Tract 219, Lake County, Indiana",155,0,<NA>,69,...,65,0,0,0,135,<NA>,<NA>,<NA>,<NA>,11825.125000


In [10]:
data1 = data.dropna()

In [11]:
data1

,uu_id,timeperiod,week_number,countyfips,tract,tract_name,total_claims,edu_8th_or_less,edu_grades_9_11,edu_hs_grad_equiv,...,gender_male,gender_na,race_amerindian,race_asian,race_black,race_noanswer,race_hawaiiannative,race_other,race_white,average_wage
27,1bd2241076381b8c9e85d660512a013e,20220514,20,18039,1300,"Census Tract 13, Elkhart County, Indiana",16,0,0,0,...,0,0,0,0,16,0,0,0,0,17617.230769
566,81594c0c7e4ef4c4cdbb7133d01cb361,20220101,1,18067,10100,"Census Tract 101, Howard County, Indiana",28,0,0,15,...,14,0,0,0,0,0,0,0,28,12577.428571
1483,6f483941cceef95c97d28b4f904a0d57,20220129,5,18127,51002,"Census Tract 510.02, Porter County, Indiana",37,0,0,14,...,21,0,0,0,0,0,0,0,37,13228.769231
3634,06c78e49b4daedfeb808c42e58fb25e4,20220402,14,18063,210603,"Census Tract 2106.03, Hendricks County, Indiana",45,0,0,21,...,23,0,0,0,19,0,0,0,26,14976.000000
7094,9e943f3dccee65ed33733a794a4d030e,20220115,3,18097,321600,"Census Tract 3216, Marion County, Indiana",37,0,0,17,...,15,0,0,0,26,0,0,0,11,9925.714286
8167,0ad94f09274e2c9cb0ef5cb77eb334b4,20220903,36,18097,380100,"Census Tract 3801, Marion County, Indiana",73,0,11,42,...,18,0,0,0,17,0,0,0,56,13343.217391
8169,0ad94f09274e2c9cb0ef5cb77eb334b4,20220903,36,18097,380100,"Census Tract 3801, Marion County, Indiana",73,0,11,42,...,18,0,0,0,17,0,0,0,56,13343.217391
8586,e50bc1df1b2b46879a29c009cf17654e,20220115,3,18061,60600,"Census Tract 606, Harrison County, Indiana",31,0,0,21,...,18,0,0,0,0,0,0,0,31,7219.200000
8917,7e2478517261a19d87c0ab2e737fb25f,20220312,11,18163,1000,"Census Tract 10, Vanderburgh County, Indiana",53,0,0,32,...,38,0,0,0,12,0,0,0,41,9706.666667
10856,a90462cd11ae4e43144239bf7c4828a4,20220129,5,18153,50300,"Census Tract 503, Sullivan County, Indiana",37,0,10,17,...,25,0,0,0,0,0,0,0,37,10040.857143


In [12]:
data.head()

,uu_id,timeperiod,week_number,countyfips,tract,tract_name,total_claims,edu_8th_or_less,edu_grades_9_11,edu_hs_grad_equiv,...,gender_male,gender_na,race_amerindian,race_asian,race_black,race_noanswer,race_hawaiiannative,race_other,race_white,average_wage
0,f013068de98db1470bd986137a0c6d23,20220416,16,18003,900,"Census Tract 9, Allen County, Indiana",22,0,<NA>,14,...,<NA>,0,0,0,11,0,0,<NA>,<NA>,8347.125000
1,21957d5517323845818d87623589e1ba,20220319,12,18089,10400,"Census Tract 104, Lake County, Indiana",111,0,<NA>,108,...,35,0,0,0,<NA>,<NA>,0,0,0,7036.636364
2,6a5609f385912113b6f1014b958ed748,20220326,13,18089,11500,"Census Tract 115, Lake County, Indiana",39,0,<NA>,<NA>,...,<NA>,0,0,0,39,0,0,0,0,7890.142857
3,46b2882ec4c373527ec33f7bd4f1388d,20220716,29,18089,20700,"Census Tract 207, Lake County, Indiana",14,0,<NA>,<NA>,...,<NA>,0,0,<NA>,10,0,0,<NA>,<NA>,7534.375000
4,37495d17e82f7df326bfc2c4c090f7b7,20220409,15,18089,21900,"Census Tract 219, Lake County, Indiana",155,0,<NA>,69,...,65,0,0,0,135,<NA>,<NA>,<NA>,<NA>,11825.125000
